# Défi Grosses Data 2018 - Réseau de neurones avec scikit-learn

In [8]:
import numpy as np
import pandas as pd
import datetime as dt
from os import listdir
import imp
import Annex
imp.reload(Annex)
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [3]:
X_train,X_test,Y_train,Y_test,X,Y,scaler=Annex.get_data_raw(scale=True, 
                                                        add_dummies=True,
                                                        var_dummies=['insee','ddH10_rose4'],
                                                        TrainTestSplit=True,
                                                        sz_test=0.3,
                                                        impute_method='drop',
                                                        convert_month2int=True,
                                                        date_method='week_number')#'drop'

We are addressing your request.
Data has been imported. Size: (189280, 31)
Months converted to int.
Dummies added.
Date transformed in week number.
26528 data points deleted. 14.02 %
Train size: 113926, Test size: 48826
Data scaled


In [ ]:
mlp=MLPRegressor(max_iter=500,hidden_layer_sizes =(80,80))
nnetOpt=mlp.fit(X_train, Y_train)

In [ ]:
# prévision de l'échantillon test
Y_pred = nnetOpt.predict(X_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

In [ ]:
Annex.generate_submission_file('submission_test.csv', 
                         nnetOpt, 
                         scaler, 
                         add_dummies=True, 
                         var_dummies=['insee','ddH10_rose4'], 
                         convert_month2int=True, 
                         date_method='week_number', 
                         fillna_method='zeros' )

### Sammy, optimize!

In [4]:
import sys
sys.path.append("./../")
import Sammy
import pickle
import os
path='./results/Resultats.pickle'
if os.path.exists(path):
    Resultats=Sammy.Results()#Resultats=pickle.load(open(path, 'rb'))
else:
    Resultats=Sammy.Results()
Resultats.say_hello()

Welcome, I am Sammy Nimizz.
May I help you?


In [ ]:
Resultats.random_start(X_train,Y_train,nb_run=10,methods=XGBRegressor)
f=open('./results/Resultats.pickle', 'wb')
pickle.dump(Resultats,f)
f.close()

Run 1/10
Running XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 150, 'min_child_weight': 1, 'max_depth': 13, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
Score: 9.766e-01

Run 2/10
Running XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 250, 'min_child_weight': 1, 'max_depth': 12, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}


In [ ]:
# prévision de l'échantillon test
Y_pred = nnetOpt.predict(X_test)
print("MSE =",mean_squared_error(Y_test,Y_pred))
print("R2 =",r2_score(Y_test,Y_pred))

In [ ]:
print(Annex.Results(Resultats.bests(nb=5)))

In [10]:
Resultats.test(X_train, Y_train, X_test, Y_test, nb_test=5)

Testing XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 6, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
Test Score= 1.10067369209
Testing XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_jobs': 4, 'n_estimators': 50, 'min_child_weight': 1, 'max_depth': 5, 'max_delta_step': 0, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 1, 'colsample_bylevel': 1, 'booster': 'gbtree', 'base_score': 0.5}
Test Score= 1.34961507421
Testing XGBRegressor with parameters: {'subsample': 1, 'silent': True, 'scale_pos_weight': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'objective': 'reg:linear', 'nthread': None, 'n_job